In [ ]:
!pip install warcio
# !pip install lxml
!pip install fasttext
# !pip install langdetect
# !pip install nltk
# !pip install cchardet 
#!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 KB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import warcio
import pandas as pd
import io
import gzip
from tqdm import tqdm
from bs4 import BeautifulSoup

# Open the WARC file
filepath = '/content/gdrive/MyDrive/Common_Crawl/data/CC-NEWS-20220603044810-01543.warc.gz'
with gzip.open(filepath, 'rb') as warc_file:

    # Create an empty list to hold the data we extract
    data = []

    # Loop through each WARC record in the file and display a progress bar
    for record in tqdm(warcio.ArchiveIterator(warc_file), unit='record'):

        # Check if the record is a response to an HTTP request
        if record.rec_type == 'response':

            # Extract the URL, encoding, and language from the record's metadata
            url = record.rec_headers.get_header('WARC-Target-URI')
            content_type = record.http_headers.get('Content-Type', '')
            encoding = content_type.split('charset=')[-1]
            lang = record.rec_headers.get_header('WARC-Language')

            # Extract the raw HTML and the text from the response
            content = record.content_stream().read()
            html = ''
            text = ''
            if content_type.startswith('text/html'):
                try:
                    # Try to decode the content using the specified encoding
                    html = content.decode(encoding, 'replace')
                except LookupError:
                    # If the specified encoding is not found, use a fallback encoding
                    html = content.decode('iso-8859-1', 'replace')
                if html:
                    # Use BeautifulSoup to extract the text from the HTML
                    soup = BeautifulSoup(html, 'html.parser')
                    text = soup.get_text().strip()

                    # Extract the language from the HTML if available
                    if soup.html:
                        lang = soup.html.get('lang', lang)

            # Add the data to our list
            data.append((url, html, text, encoding, lang))

# Create a pandas dataframe from the data
df = pd.DataFrame(data, columns=['url', 'html', 'text', 'encoding', 'lang'])


62513record [20:27, 50.93record/s]


In [ ]:
import fasttext
import pandas as pd

# Load the pre-trained fasttext language detection model
model = fasttext.load_model('/content/gdrive/MyDrive/Common_Crawl/data/lid.176.bin')


# Define a function to detect the language of a given text using fasttext
def detect_language(text, lang):
    # Remove any newline characters from the text
    text = text.replace('\n', '')

    # Predict the language using the fasttext model
    predictions = model.predict(text)
    return predictions[0][0].replace('__label__', '')



# Apply the detect_language function to each row of the dataframe
df['language'] = df.apply(lambda row: detect_language(row['text'], row['lang']), axis=1)


In [ ]:
import re

# Define a function to remove special characters from the input text
def remove_special_chars(text):
    # Define the pattern to match special characters
    pattern = r'[^a-zA-z0-9\s]'

    # Replace special characters with a space
    text = re.sub(pattern, ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# Apply the remove_special_chars function to the text column
df['text'] = df['text'].apply(remove_special_chars)


df.sample(10)

,url,html,text,encoding,lang,language
10185,http://qrt.vn/khi-tuong-thuy-van/tin-du-bao-na...,"<!DOCTYPE html>\n<html lang=""vi-VN"">\n<head>\n...",ms viewport width device width zoom 1 min zoom...,UTF-8,vi-VN,vi
22339,https://hindi.sportskeeda.com/wwe/wwe-interest...,"<!DOCTYPE html>\n<html lang=""hi"">\n<head>\n<!-...",var gtmFired false document addEventListener m...,utf-8,hi,en
28342,https://theenterpriseleader.com/2022/06/03/tra...,"<!doctype html>\n<html lang=""en"">\n<head>\n<me...",if self top document documentElement style dis...,UTF-8,en,en
19626,https://economictimes.indiatimes.com/news/poli...,"<!DOCTYPE html><html xmlns:xslthelper=""com.tim...",mohan bhagwat RSS chief Mohan Bhagwat on Gyanv...,UTF-8,en,en
22552,https://www.homify.ru/specialisty/dizaynery-in...,"<!DOCTYPE html>\n<html class=""-logged-out- -lo...",function w d s l i w[l] w[l] [] w[l] push gtm ...,utf-8,ru,en
27825,https://www.dailyherald.com/article/20220602/s...,"\r\n\r\n\r\n<!doctype html>\r\n<html lang=""en-...",Schwindel Contreras and Happ homer as Cubs top...,iso-8859-1,en-US,en
5009,https://www.homify.sg/professionals/other-busi...,"<!DOCTYPE html>\n<html class=""-logged-out- -lo...",function w d s l i w[l] w[l] [] w[l] push gtm ...,utf-8,en,en
3748,https://www.lamontagne.fr/saint-hilaire-peyrou...,<!DOCTYPE HTML>\n\t\t\t<html><head>\n\t\n\t\t\...,window didomiConfig app vendors didomi [ Ask D...,UTF-8,None,en
1490,https://www.watson.ch/fr/divertissement/elizab...,\n<!DOCTYPE html>\n<!--[if (lt IE 9)]> <htm...,window watson baseUrl \ fr var dataLayer [ pla...,UTF-8,fr,en
9941,https://www.homify.pl/profesjonalisci/architek...,"<!DOCTYPE html>\n<html class=""-logged-out- -lo...",function w d s l i w[l] w[l] [] w[l] push gtm ...,utf-8,pl,en


In [ ]:
# Define a function to count the number of words in the input text
def count_words(text):
    words = text.split()
    return len(words)

# Apply the count_words function to the preprocessed_text column
df['num_words'] = df['text'].apply(count_words)


In [ ]:
# Find the maximum value in the num_words column
max_words = df['num_words'].max()

# Print the maximum value
print('Maximum number of words:', max_words)


Maximum number of words: 158120


In [ ]:
df.columns

Index(['url', 'html', 'text', 'encoding', 'lang', 'language', 'num_words'], dtype='object')

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

def get_title(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.title.text if soup.title else ''

tqdm.pandas()  # Enable tqdm for pandas

df['title'] = df['html'].progress_apply(get_title)


100%|██████████| 31256/31256 [19:41<00:00, 26.46it/s]


In [ ]:
sd= pd.read_parquet('/content/gdrive/MyDrive/Common_Crawl/data/dataset.parquet')
sd.head()

,url,html,text,encoding,lang,language,num_words,title
0,https://www.pulse.ng/sports/opinion/3-things-l...,"\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"">\n\...",CDATA ringDataLayer context variant accelerato...,utf-8,en,en,4797,3 things learnt from Jose Peseiro's second con...
1,https://sportspyder.com/cf/florida-gators-foot...,"<!doctype html><html lang=""en""><head><meta cha...",Sports News Tweets Rumors and Articles SportSp...,UTF-8,en,en,1611,"Sports News, Tweets, Rumors and Articles | Spo..."
2,https://www.homify.hk/professionals/kitchen-pl...,"<!DOCTYPE html>\n<html class=""-logged-out- -lo...",function w d s l i wl wl wl push gtm start ne...,utf-8,en,en,1334,Find the right Kitchen Planners in İstanbul | ...
3,"https://wyborcza.pl/7,75399,28495755,100-dni-w...",\n<!DOCTYPE html>\n<html>\n<head>\n<meta chars...,Wyborcza pl body font family Arial sans serif ...,UTF-8,None,en,689,Wyborcza.pl
4,https://www.st.nu/2022-06-03/byggforetagen-vil...,"<!DOCTYPE html><html lang=""sv"" id=""root-elemen...",window hdsconfig androidAppPackage se mittmedi...,utf-8,sv,sv,2956,Byggföretagen vill mörka lönedumpning – Sundsv...
